### Cell-cell interaction analysis of MUC6 cells
Want to analyse cell-cell interactions with CellPhoneDB
- with full counts disease MUC6 cells vs all
- with less counts (more cells) disease MUC6 cells vs all - THIS NOTEBOOK
--> in all cases downsample to 50 cells per donor per cell type - analysis should be only on adult small intestine

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import sys
import os
from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import ast

In [3]:
import random

In [4]:
sc.logging.print_header()

scanpy==1.7.1 anndata==0.8.0 umap==0.4.6 numpy==1.20.1 scipy==1.6.1 pandas==1.2.3 scikit-learn==0.24.1 statsmodels==0.12.2 python-igraph==0.8.3 louvain==0.7.0 leidenalg==0.8.3


In [5]:
ad1 = sc.read_h5ad('/nfs/team205/ao15/Megagut/Annotations_v3/h5ad/pooled_healthy_disease.plus_additional_epi.18485genes.with_fineannot.nodoublets.20230322.h5ad')

In [6]:
ad1

AnnData object with n_obs × n_vars = 1596203 × 18485
    var: 'gene_ids-0', 'feature_type-0', 'mito-0', 'ribo-0', 'hb-0', 'cc-0', 'ig-0', 'tcr-0', 'n_counts-0-0', 'n_counts_raw-0-0', 'n_counts_spliced-0-0', 'n_counts_unspliced-0-0', 'n_cells-0-0', 'n_cells_raw-0-0', 'n_cells_spliced-0-0', 'n_cells_unspliced-0-0', 'n_counts-1-0', 'n_counts_raw-1-0', 'n_counts_spliced-1-0', 'n_counts_unspliced-1-0', 'n_cells-1-0', 'n_cells_raw-1-0', 'n_cells_spliced-1-0', 'n_cells_unspliced-1-0', 'gene_ids-0-0-1', 'feature_type-0-0-1', 'mito-0-0-1', 'ribo-0-0-1', 'hb-0-0-1', 'n_counts-0-0-1', 'n_counts_raw-0-0-1', 'n_counts_spliced-0-0-1', 'n_counts_unspliced-0-0-1', 'n_cells-0-0-1', 'n_cells_raw-0-0-1', 'n_cells_spliced-0-0-1', 'n_cells_unspliced-0-0-1', 'cc-0-0-1', 'ig-0-0-1', 'tcr-0-0-1', 'gene_id-1-0-1', 'mt-1-0-1', 'ensmbl_id-colon-1-0-1', 'n_cells_by_counts-colon-1-0-1', 'mean_counts-colon-1-0-1', 'pct_dropout_by_counts-colon-1-0-1', 'total_counts-colon-1-0-1', 'n_cells_by_counts-ileum-1-0-1', 'mea

In [7]:
#function for downsampling
def downsample_mask(clustering, to_thin, n_cells=200):
    mask = np.array([i not in to_thin for i in clustering])
    numtemp = np.arange(len(clustering))
    for clus in to_thin:
        inds = set(numtemp[[i==clus for i in clustering]])
        keep = random.sample(inds, round(n_cells))
        mask[keep] = True
    return mask

In [8]:
ad1.obs = pd.read_csv('/nfs/team205/ao15/Megagut/Annotations_v3/metadata/pooled_healthy_disease.plus_additional_epi.18485genes.with_fineannot.nodoublets.20230322.csv',index_col=0)

/home/jovyan/my-conda-envs/standard/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,5,6,8,9,11,12,14,62,63,64,65,66,67,70,73,75,82,83,84,86,92,93,95,100,101,102,105,106,108,109,111,112,113,114,115,117,119,123,124,125,126,127,128,129,130,131,132,133) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
ad1

AnnData object with n_obs × n_vars = 1596203 × 18485
    obs: 'latent_cell_probability', 'latent_RT_efficiency', 'cecilia22_predH', 'cecilia22_predH_prob', 'cecilia22_predH_uncertain', 'cecilia22_predL', 'cecilia22_predL_prob', 'cecilia22_predL_uncertain', 'elmentaite21_pred', 'elmentaite21_pred_prob', 'elmentaite21_pred_uncertain', 'suo22_pred', 'suo22_pred_prob', 'suo22_pred_uncertain', 'n_counts', 'log1p_n_counts', 'n_genes', 'log1p_n_genes', 'percent_mito', 'n_counts_mito', 'percent_ribo', 'n_counts_ribo', 'percent_hb', 'n_counts_hb', 'percent_top50', 'n_counts_raw', 'log1p_n_counts_raw', 'n_genes_raw', 'log1p_n_genes_raw', 'percent_mito_raw', 'n_counts_mito_raw', 'percent_ribo_raw', 'n_counts_ribo_raw', 'percent_hb_raw', 'n_counts_hb_raw', 'percent_top50_raw', 'n_counts_spliced', 'log1p_n_counts_spliced', 'n_genes_spliced', 'log1p_n_genes_spliced', 'percent_mito_spliced', 'n_counts_mito_spliced', 'percent_ribo_spliced', 'n_counts_ribo_spliced', 'percent_hb_spliced', 'n_counts_hb_s

In [10]:
ad1.obs.disease.value_counts()

control                   563547
inutero                   333624
neighbouring_inflammed    210910
neighbouring_cancer       152292
cancer_gastric            101594
crohns_disease             75779
pediatric_IBD              67092
ulcerative_colitis         35891
cancer_colorectal          28684
neighbouring_polyps        15648
preterm                     9512
celiac_active                896
celiac_treated               734
Name: disease, dtype: int64

In [11]:
ad1 = ad1[~ad1.obs.disease.isin(['inutero','preterm'])].copy()

In [12]:
ad1.obs.disease.value_counts()

control                   563547
neighbouring_inflammed    210910
neighbouring_cancer       152292
cancer_gastric            101594
crohns_disease             75779
pediatric_IBD              67092
ulcerative_colitis         35891
cancer_colorectal          28684
neighbouring_polyps        15648
celiac_active                896
celiac_treated               734
Name: disease, dtype: int64

In [13]:
ad1.X.max()

144214.0

In [14]:
sc.pp.normalize_per_cell(ad1, counts_per_cell_after=1e4)
#sc.pp.log1p(ad1) no log transform for cellphoneDB - normalising only!

In [15]:
ad1.X.max()

10000.0

In [16]:
#Prepare categories for downsampling (per donor_celltype combination)
ad1.obs['donor_celltype']=ad1.obs['donorID_unified'].astype(str)+'_'+ad1.obs['level_3_annot'].astype(str)
ad1.obs.donor_celltype.value_counts()

D207_Enterocyte                22733
D55_Keratinocyte_stratified    14295
D56_Keratinocyte_stratified    11511
D58_Keratinocyte_stratified    10007
D207_TA                         9010
                               ...  
D47_Goblet                         1
D148_Goblet                        1
D15_Macrophage_CD5L                1
D138_B_plasma_IgA2                 1
D13_B_preB                         1
Name: donor_celltype, Length: 8599, dtype: int64

In [17]:
#Downsample to 50 cells per cell type per donor
vc=ad1.obs.donor_celltype.value_counts().loc[lambda x: x>50].reset_index()['index']
mask_p = downsample_mask(ad1.obs['donor_celltype'], vc.tolist(), n_cells=50)
ad1 = ad1[mask_p]
ad1.obs.donor_celltype.value_counts()

D21_B_memory              50
D211_Tuft                 50
D115_B_naive              50
D118_Tfh                  50
D117_Mucous_gland_neck    50
                          ..
D6_Follicular_DC           1
D97_Mature_colonocyte      1
D18_EC_capillary           1
D17_Tuft                   1
D135_Mature_colonocyte     1
Name: donor_celltype, Length: 8599, dtype: int64

In [18]:
ad1

View of AnnData object with n_obs × n_vars = 218039 × 18485
    obs: 'latent_cell_probability', 'latent_RT_efficiency', 'cecilia22_predH', 'cecilia22_predH_prob', 'cecilia22_predH_uncertain', 'cecilia22_predL', 'cecilia22_predL_prob', 'cecilia22_predL_uncertain', 'elmentaite21_pred', 'elmentaite21_pred_prob', 'elmentaite21_pred_uncertain', 'suo22_pred', 'suo22_pred_prob', 'suo22_pred_uncertain', 'n_counts', 'log1p_n_counts', 'n_genes', 'log1p_n_genes', 'percent_mito', 'n_counts_mito', 'percent_ribo', 'n_counts_ribo', 'percent_hb', 'n_counts_hb', 'percent_top50', 'n_counts_raw', 'log1p_n_counts_raw', 'n_genes_raw', 'log1p_n_genes_raw', 'percent_mito_raw', 'n_counts_mito_raw', 'percent_ribo_raw', 'n_counts_ribo_raw', 'percent_hb_raw', 'n_counts_hb_raw', 'percent_top50_raw', 'n_counts_spliced', 'log1p_n_counts_spliced', 'n_genes_spliced', 'log1p_n_genes_spliced', 'percent_mito_spliced', 'n_counts_mito_spliced', 'percent_ribo_spliced', 'n_counts_ribo_spliced', 'percent_hb_spliced', 'n_coun

In [19]:
ad1.obs.organ_groups.value_counts()

Small_intestine    67876
Large_intestine    65018
Stomach            50562
Oral_mucosa        23181
Oesophagus          5963
Salivary_gland      5439
Name: organ_groups, dtype: int64

In [20]:
#filter to cells in small intestine only
ad1 = ad1[ad1.obs.organ_groups.isin(['Small_intestine'])].copy()

In [21]:
ad1

AnnData object with n_obs × n_vars = 67876 × 18485
    obs: 'latent_cell_probability', 'latent_RT_efficiency', 'cecilia22_predH', 'cecilia22_predH_prob', 'cecilia22_predH_uncertain', 'cecilia22_predL', 'cecilia22_predL_prob', 'cecilia22_predL_uncertain', 'elmentaite21_pred', 'elmentaite21_pred_prob', 'elmentaite21_pred_uncertain', 'suo22_pred', 'suo22_pred_prob', 'suo22_pred_uncertain', 'n_counts', 'log1p_n_counts', 'n_genes', 'log1p_n_genes', 'percent_mito', 'n_counts_mito', 'percent_ribo', 'n_counts_ribo', 'percent_hb', 'n_counts_hb', 'percent_top50', 'n_counts_raw', 'log1p_n_counts_raw', 'n_genes_raw', 'log1p_n_genes_raw', 'percent_mito_raw', 'n_counts_mito_raw', 'percent_ribo_raw', 'n_counts_ribo_raw', 'percent_hb_raw', 'n_counts_hb_raw', 'percent_top50_raw', 'n_counts_spliced', 'log1p_n_counts_spliced', 'n_genes_spliced', 'log1p_n_genes_spliced', 'percent_mito_spliced', 'n_counts_mito_spliced', 'percent_ribo_spliced', 'n_counts_ribo_spliced', 'percent_hb_spliced', 'n_counts_hb_spl

In [22]:
df = pd.DataFrame(ad1.X.todense(), index=ad1.obs_names, columns=ad1.var_names)
#tranpose
df = df.T
df

index,AAAGCAACATACAGCT-HCA_A_GT12934998,AACCATGCAGCTGTTA-HCA_A_GT12934998,ACATCAGAGCTCCCAG-HCA_A_GT12934998,ACATCAGGTTACGTCA-HCA_A_GT12934998,ACCCACTGTATAATGG-HCA_A_GT12934998,ACCTTTACACTCGACG-HCA_A_GT12934998,ACGAGGATCGGTCTAA-HCA_A_GT12934998,ACGGCCACAATCCAAC-HCA_A_GT12934998,ACTGCTCAGTCCAGGA-HCA_A_GT12934998,AGCATACGTTACTGAC-HCA_A_GT12934998,...,TGGGCGTGTCGAATCT-1_16-GI4401_DUO_EPI_GEX,TGGTTCCCATTGGGCC-1_16-GI4401_DUO_EPI_GEX,TGTGTTTTCTGTTTGT-1_16-GI4401_DUO_EPI_GEX,TTCGAAGGTGGCTCCA-1_16-GI4401_DUO_EPI_GEX,TTCTACATCAGCATGT-1_16-GI4401_DUO_EPI_GEX,TTGGAACAGCTAACAA-1_16-GI4401_DUO_EPI_GEX,TTGTAGGTCATGCAAC-1_16-GI4401_DUO_EPI_GEX,TTTATGCAGTAGTGCG-1_16-GI4401_DUO_EPI_GEX,TTTATGCTCTTGCCGT-1_16-GI4401_DUO_EPI_GEX,TTTCCTCGTAGCGATG-1_16-GI4401_DUO_EPI_GEX
AL627309.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
FAM87B,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
LINC00115,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
FAM41C,0.0,1.242236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
SAMD11,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145212.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AC011043.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.324679
AC011841.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AL354822.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [23]:
df.to_csv('/nfs/team205/ao15/Megagut/Annotations_v3/disease_analysis/interactions_cellphoneDB/counts/pooled_healthy_disease.remapped.18485genes.AP_SI.counts_normalisedonly.csv')

In [24]:
meta = ad1.obs.level_3_annot
meta

index
AAAGCAACATACAGCT-HCA_A_GT12934998                         Tnaive/cm_CD8
AACCATGCAGCTGTTA-HCA_A_GT12934998                         Tnaive/cm_CD4
ACATCAGAGCTCCCAG-HCA_A_GT12934998                              Monocyte
ACATCAGGTTACGTCA-HCA_A_GT12934998                               NK_CD16
ACCCACTGTATAATGG-HCA_A_GT12934998                         B_plasmablast
                                                       ...             
TTGGAACAGCTAACAA-1_16-GI4401_DUO_EPI_GEX                             TA
TTGTAGGTCATGCAAC-1_16-GI4401_DUO_EPI_GEX                             TA
TTTATGCAGTAGTGCG-1_16-GI4401_DUO_EPI_GEX    BEST4_enterocyte_colonocyte
TTTATGCTCTTGCCGT-1_16-GI4401_DUO_EPI_GEX                Enteroendocrine
TTTCCTCGTAGCGATG-1_16-GI4401_DUO_EPI_GEX                Enteroendocrine
Name: level_3_annot, Length: 67876, dtype: object

In [25]:
meta.to_csv('/nfs/team205/ao15/Megagut/Annotations_v3/disease_analysis/interactions_cellphoneDB/meta/pooled_healthy_disease.remapped.18485genes.AP_SI.meta.csv')

In [26]:
ad1.obs.disease.value_counts()

control                   22233
neighbouring_inflammed    17320
crohns_disease            12288
pediatric_IBD              9910
neighbouring_polyps        4524
neighbouring_cancer         668
celiac_active               530
celiac_treated              403
Name: disease, dtype: int64

In [27]:
healthy = ad1[ad1.obs.disease.isin(['control'])].copy()

In [28]:
disease = ad1[ad1.obs.disease.isin(['crohns_disease','pediatric_IBD','celiac_active'])].copy()

In [29]:
healthy

AnnData object with n_obs × n_vars = 22233 × 18485
    obs: 'latent_cell_probability', 'latent_RT_efficiency', 'cecilia22_predH', 'cecilia22_predH_prob', 'cecilia22_predH_uncertain', 'cecilia22_predL', 'cecilia22_predL_prob', 'cecilia22_predL_uncertain', 'elmentaite21_pred', 'elmentaite21_pred_prob', 'elmentaite21_pred_uncertain', 'suo22_pred', 'suo22_pred_prob', 'suo22_pred_uncertain', 'n_counts', 'log1p_n_counts', 'n_genes', 'log1p_n_genes', 'percent_mito', 'n_counts_mito', 'percent_ribo', 'n_counts_ribo', 'percent_hb', 'n_counts_hb', 'percent_top50', 'n_counts_raw', 'log1p_n_counts_raw', 'n_genes_raw', 'log1p_n_genes_raw', 'percent_mito_raw', 'n_counts_mito_raw', 'percent_ribo_raw', 'n_counts_ribo_raw', 'percent_hb_raw', 'n_counts_hb_raw', 'percent_top50_raw', 'n_counts_spliced', 'log1p_n_counts_spliced', 'n_genes_spliced', 'log1p_n_genes_spliced', 'percent_mito_spliced', 'n_counts_mito_spliced', 'percent_ribo_spliced', 'n_counts_ribo_spliced', 'percent_hb_spliced', 'n_counts_hb_spl

In [30]:
disease

AnnData object with n_obs × n_vars = 22728 × 18485
    obs: 'latent_cell_probability', 'latent_RT_efficiency', 'cecilia22_predH', 'cecilia22_predH_prob', 'cecilia22_predH_uncertain', 'cecilia22_predL', 'cecilia22_predL_prob', 'cecilia22_predL_uncertain', 'elmentaite21_pred', 'elmentaite21_pred_prob', 'elmentaite21_pred_uncertain', 'suo22_pred', 'suo22_pred_prob', 'suo22_pred_uncertain', 'n_counts', 'log1p_n_counts', 'n_genes', 'log1p_n_genes', 'percent_mito', 'n_counts_mito', 'percent_ribo', 'n_counts_ribo', 'percent_hb', 'n_counts_hb', 'percent_top50', 'n_counts_raw', 'log1p_n_counts_raw', 'n_genes_raw', 'log1p_n_genes_raw', 'percent_mito_raw', 'n_counts_mito_raw', 'percent_ribo_raw', 'n_counts_ribo_raw', 'percent_hb_raw', 'n_counts_hb_raw', 'percent_top50_raw', 'n_counts_spliced', 'log1p_n_counts_spliced', 'n_genes_spliced', 'log1p_n_genes_spliced', 'percent_mito_spliced', 'n_counts_mito_spliced', 'percent_ribo_spliced', 'n_counts_ribo_spliced', 'percent_hb_spliced', 'n_counts_hb_spl

In [31]:
df = pd.DataFrame(healthy.X.todense(), index=healthy.obs_names, columns=healthy.var_names)
#tranpose
df = df.T
df

index,AAAGCAACATACAGCT-HCA_A_GT12934998,AACCATGCAGCTGTTA-HCA_A_GT12934998,ACATCAGAGCTCCCAG-HCA_A_GT12934998,ACATCAGGTTACGTCA-HCA_A_GT12934998,ACCCACTGTATAATGG-HCA_A_GT12934998,ACCTTTACACTCGACG-HCA_A_GT12934998,ACGAGGATCGGTCTAA-HCA_A_GT12934998,ACGGCCACAATCCAAC-HCA_A_GT12934998,ACTGCTCAGTCCAGGA-HCA_A_GT12934998,AGCATACGTTACTGAC-HCA_A_GT12934998,...,TGGTTAGGTTCGCTAA-1_14-GI4394_DUO_EPI_GEX,TGTTCCGCAAGAGGCT-1_14-GI4394_DUO_EPI_GEX,TGTTCCGTCCTTGACC-1_14-GI4394_DUO_EPI_GEX,TTAGGACTCACAATGC-1_14-GI4394_DUO_EPI_GEX,TTAGGCAGTAAACGCG-1_14-GI4394_DUO_EPI_GEX,TTCGGTCAGACAGGCT-1_14-GI4394_DUO_EPI_GEX,TTCTACAAGGCTACGA-1_14-GI4394_DUO_EPI_GEX,TTCTTAGCATGCCTTC-1_14-GI4394_DUO_EPI_GEX,TTGCCGTCAAAGAATC-1_14-GI4394_DUO_EPI_GEX,TTTACTGAGACACGAC-1_14-GI4394_DUO_EPI_GEX
AL627309.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM87B,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LINC00115,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM41C,0.0,1.242236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SAMD11,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145212.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AC011043.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AC011841.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL354822.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
df.to_csv('/nfs/team205/ao15/Megagut/Annotations_v3/disease_analysis/interactions_cellphoneDB/counts/pooled_healthy.remapped.18485genes.AP_SI.counts_normalisedonly.csv')

In [33]:
df = pd.DataFrame(disease.X.todense(), index=disease.obs_names, columns=disease.var_names)
#tranpose
df = df.T
df

index,AAACCTGAGGCCCTCA-4918STDY7273964,AAACCTGTCCAAACTG-4918STDY7273964,AAACGGGGTAAAGGAG-4918STDY7273964,AAAGATGAGATGTTAG-4918STDY7273964,AAAGATGAGTCCAGGA-4918STDY7273964,AAAGATGCAATGAATG-4918STDY7273964,AAAGATGCATGCGCAC-4918STDY7273964,AAAGATGTCAACGAAA-4918STDY7273964,AAAGCAAAGAGCTGGT-4918STDY7273964,AAAGCAACACCATCCT-4918STDY7273964,...,TGGGCGTGTCGAATCT-1_16-GI4401_DUO_EPI_GEX,TGGTTCCCATTGGGCC-1_16-GI4401_DUO_EPI_GEX,TGTGTTTTCTGTTTGT-1_16-GI4401_DUO_EPI_GEX,TTCGAAGGTGGCTCCA-1_16-GI4401_DUO_EPI_GEX,TTCTACATCAGCATGT-1_16-GI4401_DUO_EPI_GEX,TTGGAACAGCTAACAA-1_16-GI4401_DUO_EPI_GEX,TTGTAGGTCATGCAAC-1_16-GI4401_DUO_EPI_GEX,TTTATGCAGTAGTGCG-1_16-GI4401_DUO_EPI_GEX,TTTATGCTCTTGCCGT-1_16-GI4401_DUO_EPI_GEX,TTTCCTCGTAGCGATG-1_16-GI4401_DUO_EPI_GEX
AL627309.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
FAM87B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
LINC00115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
FAM41C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
SAMD11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145212.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AC011043.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.324679
AC011841.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AL354822.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [35]:
df.to_csv('/nfs/team205/ao15/Megagut/Annotations_v3/disease_analysis/interactions_cellphoneDB/counts/pooled_disease.remapped.18485genes.AP_SI.counts_normalisedonly.csv')

In [36]:
meta = healthy.obs.level_3_annot
meta

index
AAAGCAACATACAGCT-HCA_A_GT12934998                         Tnaive/cm_CD8
AACCATGCAGCTGTTA-HCA_A_GT12934998                         Tnaive/cm_CD4
ACATCAGAGCTCCCAG-HCA_A_GT12934998                              Monocyte
ACATCAGGTTACGTCA-HCA_A_GT12934998                               NK_CD16
ACCCACTGTATAATGG-HCA_A_GT12934998                         B_plasmablast
                                                       ...             
TTCGGTCAGACAGGCT-1_14-GI4394_DUO_EPI_GEX                             TA
TTCTACAAGGCTACGA-1_14-GI4394_DUO_EPI_GEX                             TA
TTCTTAGCATGCCTTC-1_14-GI4394_DUO_EPI_GEX    BEST4_enterocyte_colonocyte
TTGCCGTCAAAGAATC-1_14-GI4394_DUO_EPI_GEX                     Enterocyte
TTTACTGAGACACGAC-1_14-GI4394_DUO_EPI_GEX                Enteroendocrine
Name: level_3_annot, Length: 22233, dtype: object

In [37]:
meta.to_csv('/nfs/team205/ao15/Megagut/Annotations_v3/disease_analysis/interactions_cellphoneDB/meta/pooled_healthy.remapped.18485genes.AP_SI.meta.csv')

In [38]:
meta = disease.obs.level_3_annot
meta

index
AAACCTGAGGCCCTCA-4918STDY7273964                              gdT_naive
AAACCTGTCCAAACTG-4918STDY7273964                          Tnaive/cm_CD4
AAACGGGGTAAAGGAG-4918STDY7273964                                B_GC_II
AAAGATGAGATGTTAG-4918STDY7273964                          Tnaive/cm_CD4
AAAGATGAGTCCAGGA-4918STDY7273964                         Goblet_cycling
                                                       ...             
TTGGAACAGCTAACAA-1_16-GI4401_DUO_EPI_GEX                             TA
TTGTAGGTCATGCAAC-1_16-GI4401_DUO_EPI_GEX                             TA
TTTATGCAGTAGTGCG-1_16-GI4401_DUO_EPI_GEX    BEST4_enterocyte_colonocyte
TTTATGCTCTTGCCGT-1_16-GI4401_DUO_EPI_GEX                Enteroendocrine
TTTCCTCGTAGCGATG-1_16-GI4401_DUO_EPI_GEX                Enteroendocrine
Name: level_3_annot, Length: 22728, dtype: object

In [39]:
meta.to_csv('/nfs/team205/ao15/Megagut/Annotations_v3/disease_analysis/interactions_cellphoneDB/meta/pooled_disease.remapped.18485genes.AP_SI.meta.csv')

### Run CPDB using CellGenIT singularity container /nfs/cellgeni/singularity/images/cellphonedb-v3.0.2.sif